In [143]:
import common
import pandas as pd
from bs4 import BeautifulSoup
import re
import requests
import os, pathlib, enum
import json
from pathlib import Path
from sqlalchemy import text
from sqlalchemy.exc import IntegrityError
import psycopg2
import matplotlib.pyplot as plt
from IPython.display import Markdown
import numpy as np
import math

database = common.Database()
pcaps = pd.read_sql("SELECT * FROM pcap", database.engine)
malwares = pd.read_sql("""
SELECT
	MW.*,
	SUM(PCAP.R) AS R,
	SUM(PCAP.U) AS U
FROM
	MALWARE AS MW
	JOIN PCAP ON MW.ID = PCAP.MALWARE_ID
                       GROUP BY MW.ID
order by u
""", database.engine).set_index("id")

In [144]:
bdn = pd.read_sql("""
SELECT
    PCAP.MALWARE_ID,
    DN.DN,
    WL.RANK_BDN,
    COUNT(*) AS APP,
    SUM(CASE WHEN M.RCODE = 3 THEN 1 ELSE 0 END) AS NX,
    COUNT(DISTINCT M.PCAP_ID) AS PCAPS,
    DN_NN1.value as epsilon
FROM
    (SELECT * FROM MESSAGE WHERE IS_R is TRUE) AS M
    JOIN PCAP ON M.PCAP_ID = PCAP.ID
    JOIN DN ON M.DN_ID = DN.ID
    JOIN WHITELIST_DN WL ON WL.DN_ID = DN.ID
    JOIN (SELECT * FROM DN_NN WHERE NN_ID=1) AS DN_NN1 ON DN.id=DN_NN1.DN_ID
    WHERE
    PCAP.INFECTED is true AND PCAP.DATASET='CTU-13'
GROUP BY
    PCAP.MALWARE_ID,
    DN.DN,
    WL.RANK_BDN,
    DN_NN1.VALUE;
""", database.engine.connect())
bdn.head()

,malware_id,dn,rank_bdn,app,nx,pcaps,epsilon
0,177,adedtppnnfmztussktlnbknkn.org,NaN,7,7,1,1.000000
1,177,aefaeamofemugdieddphebijb.org,NaN,4,4,1,1.000000
2,177,aehyfivoxckrcmnvhinxoeqpzrcwgdu.biz,NaN,7,7,1,1.000000
3,177,aemfyldumrlithbaayzhib.com,NaN,7,7,1,0.999999
4,177,aemzlxlnduaigyqpjfqdiqopnyp.info,NaN,7,7,1,1.000000


In [178]:

def dbexecute_setdga(mwdga):
    mwdga_iddga = []
    for mwid in mwdga:
        mwdga_iddga.append([mwid, mwdga[mwid]])
        pass

    dfmw = pd.DataFrame(mwdga_iddga, columns=["id", "dga"])

    for _, row in dfmw.iterrows():
        with database.engine.connect() as conn:
            try:
                conn.execute(
                    text("""
                    UPDATE MALWARE SET DGA=:dga WHERE id=:id;
                    """),
                    [{"id": int(row["id"]), "dga": int(row["dga"]) }]
                )
                conn.commit()
            except IntegrityError as e:
                try:
                    raise e.orig
                except psycopg2.errors.UniqueViolation as e:
                    pass
                except:
                    print(e)

def dbtest(mwdga):
    display(Markdown("# Test with db"))

    pcaps = pd.read_sql("SELECT * FROM pcap", database.engine)
    malwares = pd.read_sql("SELECT * FROM malware", database.engine).set_index("id")

    pcaps = pcaps[(pcaps.dataset == "CTU-13") & (pcaps.infected)]

    alls = []
    for _, row in pcaps.iterrows():
        mwid = row["malware_id"]
        alls.append(malwares.loc[mwid, "dga"] == mwdga[mwid])

    display(Markdown("Success" if all(alls) else "Failed"))
    pass

mwdga = {}


In [164]:
mwlist = pcaps[(pcaps["dataset"] == "CTU-13")][["id", "malware_id"]].groupby("malware_id").agg({"id": lambda x: x.to_list()}).sort_values(by="malware_id")
mwdga = {}


bo = []

for mwid in mwlist.index:
    df = bdn.copy()
    df["nx/app"] = df["nx"] / df["app"]

    eplb = "eps > 0.5"
    nxlb = "nx > 0.5"
    lbs = [eplb, nxlb]
    dfs = {
        eplb: df[(df.malware_id == mwid) & (df["epsilon"] > 0.5)],
        nxlb: df[(df.malware_id == mwid) & (df["nx/app"] > 0.5)]
    }
    ret = {}
    ret[("mw", "id")] = mwid # type: ignore
    # ret[("mw", "dga")] = int(malwares.loc[mwid,"dga"]) # type: ignore
    ret[("mw", "responses")] = int(malwares.loc[mwid,"r"]) # type: ignore
    ret[("mw", "uniques")] = int(malwares.loc[mwid,"u"]) # type: ignore
    for how in lbs:
        ret[(how, "count")] = dfs[how].shape[0]
        ret[(how, "app")] = dfs[how].app.sum()
        ret[(how, "nx")] = dfs[how].nx.sum()
        ret[(how, "∃nx")] = (dfs[how].nx > 0).sum()
    bo.append(ret)

df = pd.DataFrame.from_records(bo)
df.columns = pd.MultiIndex.from_tuples(df.columns)

for how in [eplb]:
    df[how, "nx/app"] = ((df[how, "nx"] / df[how, "app"]).round(1) * 10).fillna(-1).astype(int)
    df[how, "∃nx/count"] = ((df[how, "∃nx"] / df[how, "count"]).round(1) * 10).fillna(-1).astype(int)
    df[how, "%"] = ((df[how, "count"] / df["mw", "uniques"]).round(2) * 100).fillna(-1).astype(int)
df[("mw", "dga")] = int(malwares.loc[mwid,"dga"]) # type: ignore

df_overall = df.copy()

df_overall

mw                   eps > 0.5                         nx > 0.5           \
     id responses uniques     count      app       nx   ∃nx    count      app   
0     1    320297   31786         0        0        0     0        0        0   
1   177      8124    2027      2002     8010     7581  1976     1888     7734   
2   178    609424    1002       989   603236   603224   989     1002   609424   
3   179      4672    2707      2570     3947     2587  2512     2581     2664   
4   180     30806    5746      4015    11215     6160  3982     5497     8756   
5   181   4642965    1240         4    30054    10457     4      100   777207   
6   182     36220    6388        78      798        6     5      164      187   
7   183     27478     100         3     2015      892     3       49     8035   
8   184     45355     625        19      160        3     1       28    23551   
9   185      4342    1237        14       36        0     0        3        4   
10  186    417836   47316      1287    10557      315    55     1191     7227   
11  187    823665   14664         0        0        0     0        0        0   
12  189     23165     935        10      293       38     1        3       66   
13  190    210201    1002       993   208523   204765   993     1001   210198   
14  191    566055    5664      1076    10774     6231  1005     1097    13070   
15  193    242718    6305      6294   242190   234498  2790     2609   236546   
16  194    107288    3410        75     1129        2     2       32      725   
17  195   3762106    5226      3948  3750883  2171171  3946     3545  2117377   
18  196    325874      70        12   297617        0     0        0        0   
19  197      4937     886        14       75        0     0       31       31   
20  523    385906    6148      3592   380529   378054  3571     3579   381783   

                  eps > 0.5                 mw  
         nx   ∃nx    nx/app ∃nx/count    % dga  
0         0     0        -1        -1    0   2  
1      7475  1888         9        10   99   2  
2    609411  1002        10        10   99   2  
3      2662  2581         7        10   95   2  
4      8507  5497         5        10   70   2  
5    462079   100         3        10    0   2  
6       182   164         0         1    1   2  
7      5513    49         4        10    3   2  
8     15199    28         0         1    3   2  
9         4     3         0         0    1   2  
10     6915  1191         0         0    3   2  
11        0     0        -1        -1    0   2  
12       64     3         1         1    1   2  
13   206418  1001        10        10   99   2  
14    11463  1097         6         9   19   2  
15   234709  2609        10         4  100   2  
16      518    32         0         0    2   2  
17  1573702  3545         6        10   76   2  
18        0     0         0         0   17   2  
19       31    31         0         0    2   2  
20   379412  3579        10        10   57   2

In [155]:
mwlist = pcaps[(pcaps["dataset"] == "CTU-13")][["id", "malware_id"]].groupby("malware_id").agg({"id": lambda x: x.to_list()}).sort_values(by="malware_id")
mwdga = {}


def genmask(mwid, appgt10=None, nx_gt0=None, rankbdn_null=None, nx_app_gt=None, nx_app_lt=None, th=None):
    df = bdn.copy()
    df["nx/app"] = df["nx"] / df["app"]
    tmp = []
    mask = pd.Series(np.ones(df.shape[0], dtype=bool))
    if appgt10 is not None:
        tmp.append("app > 10" if appgt10 else "app <= 10")
        mask &= df.app > 10 if appgt10 else df.app <= 10
    if nx_gt0 is not None:
        tmp.append("nx > 0" if nx_gt0 else "nx == 0")
        mask &= df.nx > 0 if nx_gt0 else df.nx == 0
    if rankbdn_null is not None:
        tmp.append("rankbdn null" if rankbdn_null else " rankbdn not null")
        mask &= df.rank_bdn.isna() if rankbdn_null else (~df.rank_bdn.isna())
    if nx_app_gt is not None:
        tmp.append(f"nx/app > {nx_app_gt}")
        mask &= df["nx/app"] > nx_app_gt
    if nx_app_lt is not None:
        tmp.append(f"nx/app < {nx_app_lt}")
        mask &= df["nx/app"] < nx_app_lt
    if th is not None:
        tmp.append(f"epsilon > {th}")
        mask &= df["epsilon"] > th
    df = df[(df.malware_id == mwid) & mask]
    
    header = [f"- queries: {int(malwares.loc[mwid,"r"])}", f"- uniques: {int(malwares.loc[mwid,"u"])}"]# type: ignore
    display(Markdown(f"## Malware#{mwid} info:\n\n" + "\n".join(header)))
    display(Markdown("Domains with:\n\n" + "\n".join([ '- ' + t for t in tmp ])))
    display(Markdown("are:\n\n" + "\n".join([
        f"- count: {df.shape[0]}",
        f"- app: {df.app.sum()}",
        f"- nx: {df.nx.sum()}",
        f"- ∃nx: {(df.nx > 0).sum()}"
    ])))
    if df.shape[0]:
        display(df.sort_values(by="app", ascending=False))
        pass
    else:
        display(Markdown("> No data"))
    display(Markdown("***"))
    return df.sort_values(by="app", ascending=False)

In [179]:
mwdga = {}
for _, row in df_overall.iterrows():
    if row["mw", "id"] == 1: continue
    if row["eps > 0.5", "%"] >= 40:
        genmask(row["mw", "id"]).head()
        genmask(row["mw", "id"], th=0.5).head()
        genmask(row["mw", "id"], nx_gt0=True).head()
        genmask(row["mw", "id"], nx_gt0=False).head()
        display(Markdown("***"))
        display(Markdown("***"))
    mwdga[int(row["mw","id"])] = 1 if row["eps > 0.5", "%"] < 40 else 2
    

## Malware#177 info:

- queries: 8124
- uniques: 2027

Domains with:



are:

- count: 2026
- app: 8124
- nx: 7602
- ∃nx: 1981

,malware_id,dn,rank_bdn,app,nx,pcaps,epsilon,nx/app
257,177,dns.msftncsi.com,453160.0,29,3,1,2.874122e-07,0.103448
1765,177,www.bing.com,156.0,17,1,1,3.527413e-07,0.058824
1766,177,www.google.com,3.0,14,8,1,3.055666e-07,0.571429
1767,177,www.google.cz,3963.0,14,8,1,2.709334e-04,0.571429
1939,177,ytnjqgmnhrxeathuwkvgvcts.org,NaN,8,4,1,9.999999e-01,0.500000
...,...,...,...,...,...,...,...,...
615,177,hyamdyozzlukfvwhadlzrojxwd.biz,NaN,1,1,1,1.000000e+00,1.000000
1124,177,orlvequgmjhylautgskqcpvwvogy.com,NaN,1,1,1,1.000000e+00,1.000000
52,177,auymbfuginhvbahtxsdi.biz,NaN,1,1,1,9.999991e-01,1.000000
1133,177,oueqeywcbmvgqcbyxwvwdhydedyuc.ru,NaN,1,1,1,1.000000e+00,1.000000


***

...

are:

- count: 516
- app: 3351
- nx: 0
- ∃nx: 0

,malware_id,dn,rank_bdn,app,nx,pcaps,epsilon,nx/app
97347,523,www.aol.com,1820.0,71,0,1,1.761887e-06,0.0
94757,523,gravityboard.com,5129121.0,38,0,1,3.702423e-07,0.0
95654,523,muscanet.com,5738635.0,37,0,1,7.375691e-08,0.0
97078,523,valornet.com,5720992.0,35,0,1,3.087165e-06,0.0
94850,523,hicstofkndmprlbaayhphvg.info,NaN,33,0,1,9.999999e-01,0.0
...,...,...,...,...,...,...,...,...
95861,523,ns2.ventraip.net.au,NaN,1,0,1,7.778983e-08,0.0
95863,523,ns2184.dns.dyn.com,10307.0,1,0,1,7.034780e-08,0.0
95865,523,ns3.eis.net.au,5588533.0,1,0,1,1.274335e-06,0.0
95868,523,ns3.joosterhosting.com,NaN,1,0,1,2.918430e-07,0.0


***

***

***

In [185]:
dbexecute_setdga(mwdga=mwdga)

dbtest(mwdga=mwdga)

pd.read_sql("""
SELECT MW.*
FROM
	MALWARE AS MW
	JOIN PCAP ON MW.ID = PCAP.MALWARE_ID
    WHERE DATASET='CTU-13' AND PCAP.infected is true
order by mw.id
""", database.engine).set_index("id")

# Test with db

Success

,name,sha256,dga,info,mwclass,mwtype,md5
id,,,,,,,
177,None,None,2,None,None,None,13fbc418d5a37bdc2c10da11a6ef46ae
178,tinba,8006cbd1c70b2ed096af9c72d6fef2c3e9cb0a41685408...,2,None,None,None,14010ce6f03e0a978693424d60e34ba9
179,necurse,b3ca90d4b289d31bbdefbff7d9b42bfeb0f44f97e4738c...,2,None,None,None,1db5333a57f56c4b80bc213ed7675793
180,caphaw,9966bc568225c509c3e3b3ff9f548ba59f39908200e3e8...,2,None,None,None,24dcfdb1f46e4018500db101234f6cd7
181,worm.netsky,c8fffb2e737514c551b2d7bcaf8baa459564b059cab1a3...,1,None,None,None,3018e99857f31a59e0777396ae634a8f
181,worm.netsky,c8fffb2e737514c551b2d7bcaf8baa459564b059cab1a3...,1,None,None,None,3018e99857f31a59e0777396ae634a8f
182,None,None,1,None,None,None,35cf982449765a4f163bcf822e663f03
183,None,None,1,None,None,None,43ecaeb983683f57af842c8993e242e6
184,bunitu,002e9d6217b17923b69710792fd85765c41a0fc5406ce6...,1,None,None,None,5aeb4b21066217ea08f77ba9390f31b5
